In [44]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys

from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from mne.stats import permutation_cluster_test
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate, stats
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction
import normalization

### Add Directories/ Load Files

In [45]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT


### Select Peaks and Plot Analytic Signal

In [46]:
#Load raw fif data

test_raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Fifs',
        'test',
        'Sub045_FIF.fif'
    )
)

subID = 'Sub045'
fft_name = str(subID) + '_'
print(fft_name)

Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\test\Sub045_FIF.fif...
    Reading extended channel information
    Range : 0 ... 145611 =      0.000 ...   582.444 secs
Ready.
Sub045_


C:\Users\mathiopv\AppData\Local\Temp\ipykernel_14180\2581033189.py:3: RuntimeWarning: This filename (C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\test\Sub045_FIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  test_raw = mne.io.read_raw_fif(os.path.join(


In [47]:
reload(dat_preproc)
x = test_raw.get_data() 
x1 = x[0,:]

peakMed = 82
peakStim = 63

dat_ngam = dat_preproc.low_highpass_filter(x1, peakMed-2, peakMed+2) 
dat_subh = dat_preproc.low_highpass_filter(x1, peakStim-2, peakStim+2) 

datall = [dat_ngam, dat_subh] 
labels = ['Peak'+str(peakMed)+'Hz','Peak'+str(peakStim)+'Hz']
print(labels)

['Peak82Hz', 'Peak63Hz']


In [48]:
def window_rms(a, window_size):
  a2 = np.power(a,2)
  window = np.ones(window_size)/float(window_size)
  return np.sqrt(np.convolve(a2, window, 'valid'))

In [49]:
%matplotlib qt
sm_signal_np = np.empty(shape = (2, x1.shape[0] - 499))
sm_signal_np[:] = np.nan

fig, axes = plt.subplots(1, 1, figsize=(12, 5))
wintosmooth = 500

for idx, dat in enumerate(datall):
    hiltr = hilbert(dat)
    amplitude_envelope = np.abs(hiltr)
    zscore_sign = stats.zscore(np.squeeze(amplitude_envelope))

    sm_signal = window_rms(zscore_sign, wintosmooth)
    
    plt.plot(sm_signal, label = labels[idx])
    #plt.plot(np.arange(0,75000), amplitude_envelope, label = labels[idx]) 
    
    #axes[idx].axvline(26250, color = 'b', ls='--', lw=2, label = 'Stim On')
    #axes[idx].axvline(50250, color = 'g', ls='--', lw=2, label = 'Stim Off')
    plt.ylabel('Analytic Signal')
    plt.xlim([0, sm_signal.shape[0]])

    
    sm_signal_np[idx,:] = sm_signal

    #axes[idx].set_xticks(ticks = np.arange(0, 80000, 10000), labels = np.arange(0,320,40))
    plt.xlabel('Time [sec]')

    

plt.suptitle('Smoothing Window: 500 samples')
plt.legend(loc='upper right')

plt.show()


In [50]:
x2 = x[4, :] 
sm_stim = window_rms(x2, wintosmooth)
sm_stim1 = (sm_stim)

In [52]:
154*250

38500

In [66]:
fig, ax1 = plt.subplots(figsize = (18,6))
#plt.rcParams['font.size'] = 10
ax2 = ax1.twinx()
for idx, dat in enumerate(sm_signal_np):
    ax1.plot(sm_signal_np[idx,:], label = labels[idx], lw = 2)
ax2.plot(sm_stim1, label = 'Stimulation', color = 'grey', ls='--', lw=3, alpha = 0.4)
ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
#ax2.set_yticks(np.arange(0.5, 2.5, 0.25))
#ax2.set_yticklabels(np.arange(0.25, 2.25, 0.25))
#ax1.set_xlim(0, sm_signal_np.shape[0])
#ax1.set_xticks(np.arange(0, 100000, 20000))
#ax1.set_xticklabels(np.arange(0, 400, 80))
ax1.set_xlabel('Time [samples]')
plt.title(str(subID))
plt.show()

In [53]:
sm_analSignal = np.transpose(np.squeeze(np.array([[sm_signal_np[0]], [sm_signal_np[1]],[sm_stim1]])))
sm_analSignal_df = pd.DataFrame(sm_analSignal, 
    columns = ['Spontan','StimOn','StimVec'],
    )
print(sm_analSignal_df)

         Spontan    StimOn  StimVec
0       1.368847  0.790434      0.0
1       1.369139  0.787469      0.0
2       1.369938  0.785298      0.0
3       1.370787  0.783349      0.0
4       1.371630  0.781279      0.0
...          ...       ...      ...
145108  1.512171  1.147411      NaN
145109  1.513409  1.147936      NaN
145110  1.514641  1.148590      NaN
145111  1.515861  1.149501      NaN
145112  1.517489  1.150105      NaN

[145113 rows x 3 columns]


In [56]:
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','anal_signal/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignal',dpi = 300)
sm_analSignal_df.to_csv(str(fft_file)+str(fft_name)+'sm_analSignal.csv')

### Crop Analytic Signal to Epochs of Interest

In [57]:
sm_anal = pd.read_csv(os.path.join(
    project_path,
        'data',
        'anal_signal',
        'Sub045_sm_analSignal.csv'
))

print(sm_anal)

subID = 'Sub045'
fft_name = str(subID) + '_'
print(fft_name)

        Unnamed: 0   Spontan    StimOn  StimVec
0                0  1.368847  0.790434      0.0
1                1  1.369139  0.787469      0.0
2                2  1.369938  0.785298      0.0
3                3  1.370787  0.783349      0.0
4                4  1.371630  0.781279      0.0
...            ...       ...       ...      ...
145108      145108  1.512171  1.147411      NaN
145109      145109  1.513409  1.147936      NaN
145110      145110  1.514641  1.148590      NaN
145111      145111  1.515861  1.149501      NaN
145112      145112  1.517489  1.150105      NaN

[145113 rows x 4 columns]
Sub045_


In [58]:
%matplotlib qt

plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['Spontan'])
plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimOn'])
plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimVec'], ls='--')


In [61]:
20*250

5000

In [67]:
#CROP IT

med1 = 52
med2 = 82
sub_pt = 36600
time_on = sub_pt - 25*250
time_off = sub_pt + 15*250

sm_anal1 = sm_anal.iloc[med1*250 : med2*250]
sm_anal2 = sm_anal.iloc[time_on : time_off]
sm_toplot = pd.concat([sm_anal1, sm_anal2])
print(sm_toplot)

       Unnamed: 0   Spontan    StimOn   StimVec
13000       13000  1.102678  1.133032  0.000000
13001       13001  1.088468  1.132112  0.000000
13002       13002  1.074449  1.131211  0.000000
13003       13003  1.060656  1.130331  0.000000
13004       13004  1.047124  1.129470  0.000000
...           ...       ...       ...       ...
40345       40345  1.235583  0.906546  1.741172
40346       40346  1.236496  0.904866  1.742177
40347       40347  1.237407  0.903261  1.743181
40348       40348  1.238313  0.901729  1.744185
40349       40349  1.239211  0.900269  1.745188

[17500 rows x 4 columns]


In [69]:
# CROP ONLY THE TRANSITION E.G. BETWEEN 0.5 - 1.0mA
this_sig = sm_toplot

fig, ax1 = plt.subplots(figsize = (8,6))
ax2 = ax1.twinx()

ax1.plot(np.arange(0, this_sig.shape[0]), this_sig['Spontan'], label = 'Med-Induced')
ax1.plot(np.arange(0, this_sig.shape[0]), this_sig['StimOn'], label = 'Subharmonic')
ax2.plot(np.arange(0, this_sig.shape[0]), this_sig['StimVec'], ls='--', color = 'grey')
ax1.axvline(x = 13750, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Subharmonic On')
ax1.axvline(x = 7500, color = 'grey', lw=3, alpha = 0.2)


ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
ax1.set_xlabel('Time [samples]')
ax1.set_title(str(subID))


Text(0.5, 1.0, 'Sub045')

In [70]:
fft_fig = os.path.join(project_path, 'figures','anal_signal/')
fft_file = os.path.join(project_path, 'data','anal_signal/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignalTRANSITION',dpi = 150)
this_sig.to_csv(str(fft_file)+str(fft_name)+'sm_analSignalTRANSITION.csv')

### Average Cropped Analytic Signal and Plot

In [ ]:
med_anal = pd.read_csv(os.path.join(
    project_path,
    'data',
    'anal_signal',
    'AllMed_analSignalTRANSITION.csv'
))

stim_anal = pd.read_csv(os.path.join(
    project_path,
    'data',
    'anal_signal',
    'AllStim_analSignalTRANSITION.csv'
))

print(stim_anal)

In [ ]:
med_anal_cr = med_anal.loc[7500:17500 ,:]
med_anal_mean = np.nanmean(med_anal_cr,1)
med_anal_sem = stats.sem(med_anal_cr,1, nan_policy='omit')

stim_anal_cr = stim_anal.loc[7500:17500 ,:]
stim_anal_mean = np.nanmean(stim_anal_cr,1)
stim_anal_sem = stats.sem(stim_anal_cr,1, nan_policy='omit')


In [ ]:
25*250

In [ ]:
%matplotlib qt
plt.plot(np.arange(0,10000), med_anal_mean, label = 'Med-Induced FTG')
plt.fill_between(np.arange(0,10000), med_anal_mean-med_anal_sem, med_anal_mean+med_anal_sem, alpha = 0.2)

plt.plot(np.arange(0,10000), stim_anal_mean, color = 'red', label = 'Stim-Induced FTG')
plt.fill_between(np.arange(0,10000), stim_anal_mean-stim_anal_sem, stim_anal_mean+stim_anal_sem, color = 'red', alpha = 0.2)


plt.axvline(x = 6250, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Subharmonic On')
#plt.axvline(x = 7500, color = 'grey', lw=3, alpha = 0.2)
plt.xlim(0,10000)
plt.xticks(np.arange(0,11250,1250), labels = np.arange(-5,4,1))

plt.legend()
plt.xlabel('Time [samples]')
plt.ylabel('Z-scored Smoothed Analytic Signal')

In [ ]:
np.arange(-5,4,1)

In [ ]:
fft_fig = os.path.join(project_path, 'results')

plt.savefig(str(fft_fig)+'AVG_AnalCropped',dpi = 200)

In [ ]:
pval = 0.05  # arbitrary
dfn = 2 - 1  # degrees of freedom numerator
dfd = med_anal.shape[0] - 2  # degrees of freedom denominator
thresh = scipy.stats.f.ppf(1 - pval, dfn=dfn, dfd=dfd)  # F distribution
print(thresh)

In [ ]:
perm_nd = np.empty([6,med_anal.shape[0]])
perm_nd[:] = np.nan
perm_nd[0,:] = np.transpose(med_anal)

In [ ]:
y = np.expand_dims(med_anal, axis = 2)
y1 = np.expand_dims(stim_anal, axis = 2)


In [ ]:
#H0 = np.empty([1000,6])
T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([y, y1], n_permutations=1000,
                             threshold=None, n_jobs=None)


In [ ]:
cluster_p_values

In [ ]:
import traces
